# MTGTOP8.COM SCRAPPER 

This scrapper is modeified from the orginal by camilo-nunez on Github <br>The orginal url is: https://github.com/camilo-nunez/mtgtop8-scraper/blob/master/main_scraper.ipynb <br>This notebook is taken on Jan 14, 2021

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
import numpy as np
from tqdm import tqdm
import json
import os
import shutil
import pandas as pd

In [2]:
formats = ['ST', 'MO', 'LE', 'VI', 'PI', 'EDH']

# Tools for Scraping

## fetchMetaId

In [3]:
def fetchMetaId(fmt):
    html = urlopen('http://mtgtop8.com/format?f={0}'.format(fmt))
    bs = BeautifulSoup(html, "html.parser")
    
    meta_options = bs.find('select', {"name":"meta"}).findAll('option')
    meta_dict = {}
    for o in meta_options:
        pkg = re.findall(r"format\?f=(\w{2,3})&meta=([0-9]{2,3})", o.attrs['value'])
        if len(pkg)!=0: meta_dict[pkg[0][1]] = o.get_text()
        else: pass
        
    return meta_dict

In [4]:
fetchMetaId('ST');

## fetchFormatEvents

In [5]:
def fetchFormatEvents(fmt, mtid):
    events = []

    page = 1
    flag = True
    while flag:
        html = requests.post('http://mtgtop8.com/format?f={0}&meta={1}'.format(fmt, mtid), \
                             headers={'Content-Type': 'application/x-www-form-urlencoded'}, data = {'cp':page}).text
        bs = BeautifulSoup(html, "html.parser")

        table = bs.select('div div table tr td[width="40%"] > table')[1]
        div = table.select('tr[height="30"]')

        if len(div)!=0:
            for o in div:
                link = o.select('td a', href=True, text=True)[0]

                event = {
                    'title': link.get_text(),
                    '_id': re.findall(r"e\=(\d*)", link['href'])[0],
                    'stars': len(o.select('td[width="15%"] img[src="/graph/star.png"]')),
                    'bigstars': len(o.select('td[width="15%"] img[src="graph/bigstar.png"]')),
                    'date': o.select('td[align="right"]')[0].get_text(),
                    'format': fmt,
                    'meta_id': mtid
                }
                events.append(event)
            page+=1
        else:
            flag = False
    
    return events

In [6]:
fetchFormatEvents('ST','96')[:5];

## fetchEventInfo

In [7]:
def fetchEventInfo(evtid):
    html = urlopen('http://mtgtop8.com/event?e={0}'.format(evtid))
    bs = BeautifulSoup(html, "html.parser")
    
    data = bs.select('table div table td[align=center] div')[1].previous.strip()
    players = re.findall(r"^(\d*) players", data)[0]
    date = re.findall(r"(\d\d\/\d\d\/\d\d)$", data)[0]
    title = bs.select('.w_title td')[0].get_text()
    top_8_decks = []

    for o in bs.select('table td[width="25%"] > div > div:not([align="center"])'):
        link = o.select('div div a')[0]['href']
        deck = {
            '_id': re.findall(r"\&d\=(\d*)", link)[0],
            'result':o.select('div div[align=center]')[0].get_text(),
            'title': o.select('div div a')[0].get_text(),
            'player': o.select('div div a')[0].get_text()
        }
        top_8_decks.append(deck)
    
    return top_8_decks

In [8]:
fetchEventInfo('22539');

## fetchDeck

In [9]:
def fetchDeck(evtid, plyid):
    html = urlopen('http://mtgtop8.com/event?e={0}&d={1}'.format(evtid, plyid))
    bs = BeautifulSoup(html, "html.parser")
    
    tables = bs.select('table table table')

    main_deck = []
    for t in tables[0:-1]:
        for o in t.select('tr td div span'):
            pkg = o.parent.get_text().split()
            count = pkg[0]
            card = ' '.join(pkg[1:])
            main_deck.append((card,count))

    sideboard = []
    for o in tables[-1].select('tr td div span'):
        pkg = o.parent.get_text().split()
        count = pkg[0]
        card = ' '.join(pkg[1:])
        sideboard.append((card,count))

    deck = {
        'player': bs.select('table .chosen_tr [align=right] .topic')[0].get_text(),
        'result': bs.select('table .chosen_tr [align=center]')[0].get_text(),
        'main_deck': main_deck,
        'sideboard': sideboard
    }
    
    return deck

In [10]:
fetchDeck('25681','390867');
#json.dumps(fetchDeck('25681','390867'))

# Main Scraping

In [11]:
import pandas as pd

In [12]:
fmts = []
_ids = []
names = []

for fmt in formats:
    dict_aux = fetchMetaId(fmt)
    fmts+=[fmt]*len(dict_aux)
    _ids+=list(dict_aux.keys())
    names+=list(dict_aux.values())

all_meta_info = {'meta_id': _ids,'meta_name': names,'meta_format': fmts}

In [13]:
df_meta = pd.DataFrame.from_dict(all_meta_info)

In [14]:
df_meta.head()

,meta_id,meta_name,meta_format
0,52,Last 2 Months,ST
1,50,Last 2 Weeks,ST
2,46,Large Events Last 2 Months,ST
3,210,Players Tour Online,ST
4,187,Standard 2019-2020 (M20 to Eldraine),ST


In [15]:
df_meta.to_csv('df_meta.csv')

In [16]:
all_events = []

In [17]:
meta_ids = all_meta_info['meta_id']
meta_formats = all_meta_info['meta_format']

for i in tqdm(range(len(meta_formats))):
    try:
        all_events+=fetchFormatEvents(meta_formats[i],meta_ids[i])
    except:
        print('error en: ', meta_formats[i], meta_ids[i])

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [47:07<00:00, 37.70s/it]


In [18]:
np.save('all_events.npy', np.array(all_events))

In [19]:
dict_events = {'event_title':[], 'event__id':[], 'event_stars':[], 'event_bigstars':[], 'event_date':[], 'event_format':[], 'event_meta_id':[]}
for e in all_events:
    for k,v in e.items():
        dict_events['event_'+k].append(v)

In [20]:
df_events = pd.DataFrame.from_dict(dict_events)

In [21]:
df_events.head()

,event_title,event__id,event_stars,event_bigstars,event_date,event_format,event_meta_id
0,Grinch Melee @ MTGA,28858,1,0,13/01/21,ST,52
1,HG Classic #2,28860,2,0,13/01/21,ST,52
2,Weekly Event @ The Mythic Society,28837,2,0,12/01/21,ST,52
3,Event @ Lotus eSports,28849,1,0,12/01/21,ST,52
4,Free Daily Bo1 @ Owl Central Games,28838,1,0,11/01/21,ST,52


In [22]:
df_events.to_csv('df_events.csv')

In [23]:
main_dir = './events'

In [37]:
errors_id = []
for ind in tqdm(df_events.index):
    id_event = df_events['event__id'][ind]
    try:
        path_event = os.path.join(main_dir, id_event)
        if not os.path.exists(path_event):
            os.mkdir(path_event)

            path_players_info = os.path.join(path_event, 'players_info.csv')
            event_info_players = {'player__id':[], 'player_result':[], 'player_title':[], 'player_player':[]}
            for ply in fetchEventInfo(id_event):
                for k,v in ply.items():
                    event_info_players['player_'+k].append(v)
            df_players = pd.DataFrame.from_dict(event_info_players)
            df_players.to_csv(path_players_info)

            path_players_decks = os.path.join(path_event, 'players_decks')
            os.mkdir(path_players_decks)
            for indx in df_players.index:
                id_player = df_players['player__id'][indx]
                path_player_deck = os.path.join(path_players_decks, 'player_{0}_deck.json'.format(id_player))
                with open(path_player_deck, 'w') as f:
                    json.dump(fetchDeck(id_event, id_player), f)
        else:
            pass
            #print('folder  '+ path_event+' exist')
    except:
        errors_id.append(id_event)
        print('error en: ', id_event)

  0%|                                                                            | 31/48034 [01:57<33:35:58,  2.52s/it]

error en:  28758


  0%|                                                                            | 58/48034 [03:54<53:12:29,  3.99s/it]

error en:  28620


  0%|                                                                            | 62/48034 [04:00<26:40:35,  2.00s/it]

...

 97%|████████████████████████████████████████████████████████████████████▊  | 46524/48034 [15:31:00<1:23:16,  3.31s/it]

error en:  7049


 97%|████████████████████████████████████████████████████████████████████▊  | 46544/48034 [15:32:43<1:48:13,  4.36s/it]

error en:  6828


100%|█████████████████████████████████████████████████████████████████████████| 48034/48034 [15:39:37<00:00,  1.17s/it]


In [39]:
np.save('vector_errores.npy',np.array(errors_id))

In [40]:
empy_dirs = []
for fol in os.listdir(main_dir):
    if not os.listdir(os.path.join(main_dir, fol)):
        empy_dirs.append(fol)
len(empy_dirs)

10437

In [41]:
np.save('empy_dirs.npy',np.array(empy_dirs))

In [42]:
for fol in empy_dirs:
    shutil.rmtree(os.path.join(main_dir, fol))

In [43]:
indices_errores = []
actual_events = list(map(int, os.listdir(main_dir)))
df_all_events = pd.read_csv("df_events.csv").drop_duplicates(subset ="event__id")

In [44]:
df_all_events.head()

,Unnamed: 0,event_title,event__id,event_stars,event_bigstars,event_date,event_format,event_meta_id
0,0,Grinch Melee @ MTGA,28858,1,0,13/01/21,ST,52
1,1,HG Classic #2,28860,2,0,13/01/21,ST,52
2,2,Weekly Event @ The Mythic Society,28837,2,0,12/01/21,ST,52
3,3,Event @ Lotus eSports,28849,1,0,12/01/21,ST,52
4,4,Free Daily Bo1 @ Owl Central Games,28838,1,0,11/01/21,ST,52


In [45]:
for ind in tqdm(df_all_events.index):
    id_event = int(df_all_events['event__id'][ind])
    if id_event not in actual_events:
        indices_errores.append(ind)

100%|██████████████████████████████████████████████████████████████████████████| 23135/23135 [00:03<00:00, 7581.00it/s]


In [46]:
df_all_events = df_all_events.drop(indices_errores)

In [47]:
df_all_events = df_all_events.drop(columns='Unnamed: 0')

In [48]:
df_all_events.to_csv('df_events_v2.csv', index=False)